In [1]:
%load_ext autoreload
import os

import numpy as np
import scqubits as scq
import plotly.graph_objects as go

%autoreload 2
import zcu_tools.notebook.analysis.design as zd

In [2]:
qub_name = "Design1"

os.makedirs(f"../../result/{qub_name}/image", exist_ok=True)
os.makedirs(f"../../result/{qub_name}/web", exist_ok=True)

# Scan params

In [3]:
EJb = (2.0, 7.5)
EC = 0.8
ELb = (0.35, 1.0)

flx = 0.5
r_f = 7.2

In [ ]:
Temp = 113e-3
Q_cap = 4.0e5
Q_ind = 1.7e7
x_qp = 1.5e-6

noise_channels = [
    ("t1_capacitive", dict(Q_cap=Q_cap)),
    # ("t1_inductive", dict(Q_ind=Q_ind)),
    # ("t1_quasiparticle_tunneling", dict(x_qp=x_qp)),
]

avoid_freqs = [r_f, 2 * r_f]
fluxonium = scq.Fluxonium(1.0, 1.0, 1.0, flux=flx, cutoff=40, truncated_dim=9)

params_table = zd.generate_params_table(EJb, EC, ELb, flx)
zd.calculate_esys(params_table, fluxonium)

In [ ]:
zd.calculate_f01(params_table, fluxonium)
zd.calculate_m01(params_table, fluxonium)
zd.calculate_t1(params_table, fluxonium, noise_channels, Temp)
zd.calculate_collision(params_table, avoid_freqs)
zd.calculate_dipersive_shift(params_table, g=0.1, r_f=r_f)
params_table.drop("esys", axis=1)

In [ ]:
fig = zd.plot_scan_results(params_table, xaxis="Chi")
fig.update_layout(
    title=f"T1 vs Matrix Element 0-1 (Q_cap={Q_cap:.1e}, Q_ind={Q_ind:.1e}, x_qp={x_qp:.1e})",
    title_x=0.51,
)

# params = annotate_best_point(fig, params_table)

# fmt: off
# add_real_sample(fig, "../../result/Q12_2D/Q4/params.json", t1=26.91, flx=flx, t1_info=(noise_channels, Temp))
# add_real_sample(fig, "../../result/Q12_2D/Q1/params.json", t1=31.97, flx=flx, t1_info=(noise_channels, Temp))
# add_real_sample(fig, "../../result/2DQ9/Q5/params.json", t1=14.25, flx=flx, t1_info=(noise_channels, Temp))
# fmt: on

fig.show()

In [11]:
fig.write_html(
    f"../../result/{qub_name}/web/t1vsM01_EC{EC:.2f}.html", include_plotlyjs="cdn"
)
fig.write_image(f"../../result/{qub_name}/image/t1vsM01_EC{EC:.2f}.png", format="png")

In [ ]:
params = EJ, EC, EL = 7.45, 0.8, 0.95
EJ, EC, EL = params


flxs = np.linspace(0.0, 1.0, 300)

EJ, EC, EL

In [ ]:
fluxonium = scq.Fluxonium(*params, flux=0.5, cutoff=40, truncated_dim=10)
spectrumData = fluxonium.get_matelements_vs_paramvals(
    operator="n_operator", param_name="flux", param_vals=flxs, evals_count=10
)

In [ ]:
show_idxs = [(i, j) for i in range(2) for j in range(10) if j > i]

eval_eng = spectrumData.energy_table

fig = go.Figure()
for i, j in show_idxs:
    fig.add_trace(
        go.Scatter(
            x=flxs,
            y=eval_eng[:, j] - eval_eng[:, i],
            mode="lines",
            name=f"{i} - {j}",
            line=dict(width=2),
        )
    )
for freq in avoid_freqs:
    fig.add_hline(y=freq, line_color="black", line_width=2, line_dash="dash")
fig.update_yaxes(range=[0.0, 8.0])
fig.update_layout(
    title=f"EJ/EC/EL = {EJ:.3f}/{EC:.3f}/{EL:.3f}",
    title_x=0.5,
    xaxis_title=r"$\phi_{ext}/\phi_0$",
    yaxis_title="Energy (GHz)",
)

fig.show()

In [ ]:
show_idxs = [(i, j) for i in range(2) for j in range(3) if j > i]

matrixelements = spectrumData.matrixelem_table

fig = go.Figure()
for i, j in show_idxs:
    fig.add_trace(
        go.Scatter(
            x=flxs,
            y=np.abs(matrixelements[:, i, j]),
            mode="lines",
            name=f"{i}-{j}",
            line=dict(width=2),
        )
    )
fig.update_layout(
    title=f"EJ/EC/EL = {EJ:.3f}/{EC:.3f}/{EL:.3f}",
    title_x=0.5,
    xaxis_title=r"$\phi_{ext}/\phi_0$",
    yaxis_title="Matrix elements",
)

fig.show()

In [ ]:
# Temp = 60e-3
# Q_cap = 1.0e5
# Q_ind = 1.0e7
# x_qp = 1.0e-8

scq.settings.T1_DEFAULT_WARNING = False
fluxonium = scq.Fluxonium(*params, flux=0.5, cutoff=30, truncated_dim=5)
T1_eff_fig, T1_eff_ax = fluxonium.plot_t1_effective_vs_paramvals(
    param_name="flux",
    param_vals=flxs,
    xlim=([flxs.min(), flxs.max()]),
    common_noise_options=dict(i=1, j=0, T=Temp),
    noise_channels=[
        ("t1_capacitive", dict(Q_cap=Q_cap)),
        # ("t1_inductive", dict(Q_ind=Q_ind)),
        # ("t1_quasiparticle_tunneling", dict(x_qp=x_qp)),
    ],
)
_ = T1_eff_ax.set_ylim(None, 1e6)